### XGBoost
Testing XGBoost. XGBoost provides regularization hyper parameters to guard against overfitting.
See 
- https://towardsdatascience.com/xgboost-python-example-42777d01001e
- https://towardsdatascience.com/machine-learning-part-18-boosting-algorithms-gradient-boosting-in-python-ef5ae6965be4


#### Partition
Given a tree, calculate the optimal gain (non-negative) given left-right split across a dataset.

*Gain = (G_left^2 / H_left + lambda) + (G_right^2 / H_right + lamba) - (G_right + G_left)^2/(H_left + H_right + lambda) - gamma*

- G => sum of residuals
- H => # of residuals
- lambda => regularization, redudce sensitivity to individual observations
- gamma => regularization, minimum loss reguired to advance partition


#### Calculate output at leaf-nodes
Calculate output for iterative learning.

*Output = (Sum of residuals) / (# of residuals + lambda)*

Then

*Predication = initial_predication + learning_rate * Output*


#### Learning
Use the residuals from above to create another decision and repeat the process.

The final prediction is:

*final_prediction = initial_prediction + learning_rate * prediction_1 + learning_rate * prediction_2 + learning_rate ...*


In [3]:
import pandas as pd
import xgboost as xgb
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [7]:
# Load example data into pandas

# Load california housing, apparently the boston-housing dataset has ethical issus
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

X = pd.DataFrame(housing.data, columns=housing.feature_names)
y = pd.Series(housing.target)

In [6]:
X.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [10]:
# configure xgboost
regressor = xgb.XGBRegressor(
    n_estimators=50,
    reg_lambda=1,
    gamma=0,
    max_depth=3
)

In [11]:
regressor.fit(X_train, y_train)
pd.DataFrame(regressor.feature_importances_.reshape(1, -1), columns=housing.feature_names)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,0.570981,0.065106,0.033286,0.014736,0.017554,0.154186,0.070031,0.07412


In [21]:
y_pred = regressor.predict(X_test)

In [28]:
## Check result, convert pandas series to list, zip with prediction and print out first few entries
y_test_list = y_test.to_list()

combined = zip(y_test, y_pred)
list(combined)[0: 5]

[(5.00001, 4.4592376),
 (1.42, 1.2480156),
 (1.984, 1.9429302),
 (1.546, 1.555976),
 (3.125, 3.1600575)]